<div style="text-align: center; margin-bottom: 10px; font-size: 20px; font-weight: bold;">
  Análisis del Tráfico Aéreo del Aeropuerto de San Francisco 2005-2016
</div>

<div style="text-align: center;">
  <img src="SFO.jpg" alt="SFO" style="width: 500px; height: auto;">
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Se ha solicitado realizar un análisis de la base de datos Air_Traffic_Passenger_Statistics relativa al Aeropuerto de San Francisco SFO, en el período 2005-2016, con el objeto de permitir la toma de decisiones basada en datos.

El Dataset contiene 15007 y 16 columnas, las cuales se listan a continuación.
</div>


<div style="display: flex; justify-content: center; align-items: center; flex-direction: column; text-align: center; font-size: 18px; margin-bottom: 14px;">
    <p style="margin: 0;">Descripción del Dataset</p>
    <p style="margin: 0;">El DataFrame incluye las siguiente <b>Columnas</b>:</p>
</div>

<table style="margin: 0 auto; border: 2px solid white; border-collapse: collapse; width: 30%; font-size: 12px;">
    <thead>
        <tr>
            <th style="border: 1px solid white; padding: 6px; text-align: left;">Variable</th>
            <th style="border: 1px solid white; padding: 6px; text-align: left;">Definición</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Activity Period</td>
            <td style="border: 1px solid white; padding: 6px;">Período de actividad</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Operating Airline</td>
            <td style="border: 1px solid white; padding: 6px;">Aerolínea operadora</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Operating Airline IATA Code</td>
            <td style="border: 1px solid white; padding: 6px;">Código IATA de la aerolínea operadora</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Published Airline</td>
            <td style="border: 1px solid white; padding: 6px;">Aerolínea publicada</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Published Airline IATA Code</td>
            <td style="border: 1px solid white; padding: 6px;">Código IATA de la aerolínea publicada</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">GEO Summary</td>
            <td style="border: 1px solid white; padding: 6px;">Resumen GEO</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">GEO Region</td>
            <td style="border: 1px solid white; padding: 6px;">Región GEO</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Activity Type Code</td>
            <td style="border: 1px solid white; padding: 6px;">Código de tipo de actividad</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Price Category Code</td>
            <td style="border: 1px solid white; padding: 6px;">Código de categoría de precio</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Terminal</td>
            <td style="border: 1px solid white; padding: 6px;">Terminal</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Boarding Area</td>
            <td style="border: 1px solid white; padding: 6px;">Área de embarque</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Passenger Count</td>
            <td style="border: 1px solid white; padding: 6px;">Cantidad de pasajeros</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Adjusted Activity Type Code</td>
            <td style="border: 1px solid white; padding: 6px;">Código de tipo de actividad ajustado</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Adjusted Passenger Count</td>
            <td style="border: 1px solid white; padding: 6px;">Cantidad de pasajeros ajustada</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Year</td>
            <td style="border: 1px solid white; padding: 6px;">Año</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 6px;">Month</td>
            <td style="border: 1px solid white; padding: 6px;">Mes</td>
        </tr>
    </tbody>
</table>
<style>
    @media print {
        table {
            width: 50%; /* Reduce el ancho al imprimir */
        }
        th, td {
            font-size: 9px; /* Fuente más pequeña */
            padding: 2px; /* Menos espaciado interno */
        }
        body {
            margin: 0; /* Elimina márgenes de la página al imprimir */
            padding: 0;
        }
    }
</style>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Importación de librerías
</div>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

import os
from dotenv import load_dotenv
import pandas as pd # type: ignore
import numpy as np # type: ignore
import plotly.express as px # type: ignore
import plotly.graph_objects as go # type: ignore

In [3]:
# Cargar las variables de entorno
load_dotenv()

# Obtener la ruta de entrada desde el .env
path = os.getenv("file_path")

# Obtener la ruta de salida desde el .env
output_path = os.getenv("output_path")

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
1. Crear la sección pyspark y cargar los datos
</div>

In [4]:
# Crear la sesión de Spark
spark = SparkSession.builder.appName("Air Traffic").getOrCreate()

In [5]:
# Cargar el archivo CSV
df = spark.read.csv(path, header=True, inferSchema=True)

In [6]:
# Mostrar el esquema del DataFrame
print("Esquema del DataFrame:")
df.printSchema()

Esquema del DataFrame:
root
 |-- Activity Period: integer (nullable = true)
 |-- Operating Airline: string (nullable = true)
 |-- Operating Airline IATA Code: string (nullable = true)
 |-- Published Airline: string (nullable = true)
 |-- Published Airline IATA Code: string (nullable = true)
 |-- GEO Summary: string (nullable = true)
 |-- GEO Region: string (nullable = true)
 |-- Activity Type Code: string (nullable = true)
 |-- Price Category Code: string (nullable = true)
 |-- Terminal: string (nullable = true)
 |-- Boarding Area: string (nullable = true)
 |-- Passenger Count: integer (nullable = true)
 |-- Adjusted Activity Type Code: string (nullable = true)
 |-- Adjusted Passenger Count: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: string (nullable = true)



In [7]:
# Mostrar las primeras 5 filas del DataFrame
df.show(5)

+---------------+-----------------+---------------------------+-----------------+---------------------------+-------------+----------+------------------+-------------------+----------+-------------+---------------+---------------------------+------------------------+----+-----+
|Activity Period|Operating Airline|Operating Airline IATA Code|Published Airline|Published Airline IATA Code|  GEO Summary|GEO Region|Activity Type Code|Price Category Code|  Terminal|Boarding Area|Passenger Count|Adjusted Activity Type Code|Adjusted Passenger Count|Year|Month|
+---------------+-----------------+---------------------------+-----------------+---------------------------+-------------+----------+------------------+-------------------+----------+-------------+---------------+---------------------------+------------------------+----+-----+
|         200507|     ATA Airlines|                         TZ|     ATA Airlines|                         TZ|     Domestic|        US|          Deplaned|          

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">

El resúmen estadístico muestra datos interesantes sobre la composición del dataframe.

**Columnas numéricas** 

**a. Activity Period:**
- **Mean.** El período promedio está en 2010, con una stddev de 313.3 (alrededor de 26 meses).
- **Range.** El año mínimo es julio 2005 (200507) y el máximo es marzo 2016 (201603). Un rango de casi 11 años.

**b. Passenger Count:**
- **Mean.** El promedio es de 29,240 pasajeros, con una stddev de 58,319 pasajeros, lo cual indica una gran variabilidad.
- **Range.** El mínimo es 1 pasajero y el máximo 659,837 pasajeros.
- **Percentiles.** Se observa una distribución sesgada hacia valores más bajos con algunos valores extremos altos.

**c. Adjusted Passenger Count:**
- **Mean.** El promedio es de 29,331 pasajeros, con una stddev de 58,284. Indicativo de una gran variabilidad.
- **Range.** El mínimo es 1 pasajero y el máximo 659,837 pasajeros.
- **Percentiles.** Se observa la misma distribución sesgada hacia valores más bajos con algunos valores extremos altos.

**d. Year:**
- **Mean.** 2010 es el rango promedio del dataframe, y la stddev es de 3.13 años.
- **Range.** Desde 2005 hasta 2016.

**Columnas categóricas**  
**a. Operating airline/Published airline:**
Muestra datos completos para la mayoría de los registros, pero con 54 valores faltantes (15007-14953).

**b. Geo summary:**
Muestra las categorías Domestic e International.

**c. Geo region:**
Asia mínimo y US máximo. Orden alfabético de las filas.

**d. Activity type code:**
Deplaned mínimo y Thru/Transit máximo.

**e. Price category code:**
Rango entre low fare y other. Posiblemente relacionado con clasificación de tarifas.

**f. Terminal/Boarding area:**
Desde International hasta Terminal 3, y de A a Other, lo que representa diferentes ubicaciones en el aeropuerto.

**g. Month:**
Mínimo april y máximo september.
</div>


In [8]:
# Valores estadísticos
df.summary().show()

+-------+------------------+-----------------+---------------------------+-----------------+---------------------------+-------------+----------+------------------+-------------------+-------------+-------------+------------------+---------------------------+------------------------+-----------------+---------+
|summary|   Activity Period|Operating Airline|Operating Airline IATA Code|Published Airline|Published Airline IATA Code|  GEO Summary|GEO Region|Activity Type Code|Price Category Code|     Terminal|Boarding Area|   Passenger Count|Adjusted Activity Type Code|Adjusted Passenger Count|             Year|    Month|
+-------+------------------+-----------------+---------------------------+-----------------+---------------------------+-------------+----------+------------------+-------------------+-------------+-------------+------------------+---------------------------+------------------------+-----------------+---------+
|  count|             15007|            15007|               

In [9]:
# Mostrar el total de columnas y filas del DataFrame
# Total de columnas
print(f"Total de columnas: {len(df.columns)}")

# Total de filas
print(f"Total de filas: {df.count()}")

Total de columnas: 16
Total de filas: 15007


In [10]:
# Verificar la existencia de valores nulos
null_counts = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns]).collect()
print("Conteo de valores nulos:")
for row in null_counts:
    for col_name, count_value in row.asDict().items():
        print(f"{col_name}: {count_value}")

Conteo de valores nulos:
Activity Period: 0
Operating Airline: 0
Operating Airline IATA Code: 54
Published Airline: 0
Published Airline IATA Code: 54
GEO Summary: 0
GEO Region: 0
Activity Type Code: 0
Price Category Code: 0
Terminal: 0
Boarding Area: 0
Passenger Count: 0
Adjusted Activity Type Code: 0
Adjusted Passenger Count: 0
Year: 0
Month: 0


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
2. Mostrar el total de compañías aéreas
</div>

In [11]:
# Obtener el número de aerolíneas que venden el boleto y las que operan el vuelo
num_airlines, distinct_count = [df.select(column).distinct().count() \
for column in ["Published Airline", "Operating Airline"]]

# Imprimir los resultados
print(f"Número de aerolíneas que venden los boletos: {num_airlines}")
print(f"Número de aerolíneas que operan los vuelos: {distinct_count}")

Número de aerolíneas que venden los boletos: 68
Número de aerolíneas que operan los vuelos: 77


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Existen nueve líneas aéreas que operan los vuelos en nombre otras seis aerolíneas. De ellas, SkyWest Airlines realiza los vuelos en nombre de tres operadoras distintas. 

Consideramos que United Airlines es una sola empresa, a pesar que en el DataFrame aparezca con dos identificaciones distintas: United Airlines, United Airlines - Pre 07/01/2013. Aunque podrían unificarse ambas columnas para lograr unos datos consolidados de esta compañía, el desconocimiento del por qué de la existencia de dicha separación puede llevar a la duplicación de datos y por tanto a la tergiversación de la data correspondiente. Antes de decidir como proceder hay que tener mayor información sobre este punto.
</div>

In [12]:
# Filtrar aerolíneas que no coinciden
df_diff_unique = df.filter(df["Operating Airline"] != df["Published Airline"]) \
                   .select("Operating Airline", "Published Airline") \
                   .distinct() \
                   #.toPandas()

df_diff_unique.show(truncate=False)

+---------------------------+--------------------------------+
|Operating Airline          |Published Airline               |
+---------------------------+--------------------------------+
|Air Canada Jazz            |Air Canada                      |
|SkyWest Airlines           |United Airlines                 |
|American Eagle Airlines    |American Airlines               |
|Mesa Airlines              |American Airlines               |
|SkyWest Airlines           |United Airlines - Pre 07/01/2013|
|SkyWest Airlines           |Delta Air Lines                 |
|Compass Airlines           |Delta Air Lines                 |
|Mesaba Airlines            |Delta Air Lines                 |
|Horizon Air                |Alaska Airlines                 |
|SkyWest Airlines           |Alaska Airlines                 |
|ExpressJet Airlines        |Delta Air Lines                 |
|Mesa Airlines              |US Airways                      |
|Atlantic Southeast Airlines|Delta Air Lines           

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
En el dataset se pueden observar varias columnas que presentan la misma información aunque bajo distinto enunciado. Esto genera una duplicación de datos e incrementa el consumo de recursos, por lo que hemos decidido eliminar algunas de ellas. 

En el caso particular de las columnas que presentan valores nulos, hemos decidido eliminarlas, pues no muestran información considerada relevante para el análisis.

En vista de ello se elimanan las siguientes columnas:
* <i>"Operating Airline IATA Code"
* "Published Airline IATA Code"
* "Activity Type Code"</i>

Aunque las columnas  <i>"Operating Airline"</i> y <i>"Published Airline"</i> parecen mostrar la misma información, existe una diferencia significativa entre ambas. <i>"Operating Airline"</i> es el nombre de la aerolinea que presta el servico de transporte, en tanto que <i>"Published Airline"</i> es la que vende o promociona el boleto, por lo que la información entre ambas columnas no coinciden en su totalidad, tal como se muestra en las celdas anteriores, razón por la que hemos decidido mantenerlas.

Igualmente se conservan las columnas <i>"Passenger Count"</i> y <i>"Adjusted Passenger Count"</i>, pues entre ambas existen pequeñas variaciones.
</div>

In [13]:
# Eliminar columnas y asignar el resultado nuevamente a `clean_df`
df_1 = df.drop('Operating Airline IATA Code',\
         'Published Airline IATA Code','Activity Type Code')

# Contar valores nulos en cada columna de `clean_df`
null_counts_df_1 = df_1.select([F.sum(F.col(c).isNull().cast("int")).alias(c) \
         for c in df_1.columns]).collect()

print("Conteo de valores nulos en clean_df:")
for row in null_counts_df_1:
    for col_name, count_value in row.asDict().items():
        print(f"{col_name}: {count_value}")

Conteo de valores nulos en clean_df:
Activity Period: 0
Operating Airline: 0
Published Airline: 0
GEO Summary: 0
GEO Region: 0
Price Category Code: 0
Terminal: 0
Boarding Area: 0
Passenger Count: 0
Adjusted Activity Type Code: 0
Adjusted Passenger Count: 0
Year: 0
Month: 0


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
3. Calcular el promedio de pasajeros por aerolínea para el periodo 2005-2016
</div>

In [14]:
# Calcular el promedio de pasajeros por aerolínea
average_passengers_airline = df_1.groupBy("Operating Airline").agg(
    F.round(F.mean("Adjusted Passenger Count"),2).alias("Average Passengers")
).orderBy(["Average Passengers", "Operating Airline"], ascending=[False, False])

# Mostrar el resultado
average_passengers_airline.show(truncate=False)

+--------------------------------+------------------+
|Operating Airline               |Average Passengers|
+--------------------------------+------------------+
|American Airlines               |127164.39         |
|Southwest Airlines              |81223.35          |
|Virgin America                  |74405.35          |
|United Airlines                 |72827.22          |
|Delta Air Lines                 |68515.42          |
|US Airways                      |55317.82          |
|United Airlines - Pre 07/01/2013|49365.52          |
|SkyWest Airlines                |37083.88          |
|JetBlue Airways                 |35261.14          |
|Northwest Airlines              |26205.5           |
|Compass Airlines                |23359.84          |
|Lufthansa German Airlines       |19301.97          |
|Air Canada                      |18251.56          |
|Frontier Airlines               |17787.68          |
|British Airways                 |17625.12          |
|Alaska Airlines            

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Convertir a csv
</div>

In [15]:
# Crear el path completo del archivo
output_file =os.path.join(output_path, "average_passengers_airline.csv")

# Guardar como archivo CSV usando Pandas
average_passengers_airline.toPandas()\
.to_csv(output_file, index=False)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
4. Conocer el número máximo de pasajeros por región para el período 2005-2016
</div>

In [16]:

# Agrupar por "GEO Region" y obtener el máximo de "Adjusted Passenger Count"
max_passenger_region = df_1.groupBy("GEO Region").agg(
    F.max("Adjusted Passenger Count").alias("Max Adjusted Passenger Count")
).orderBy("Max Adjusted Passenger Count", ascending=False)

# Mostrar el resultado
max_passenger_region.show(truncate=False)

+-------------------+----------------------------+
|GEO Region         |Max Adjusted Passenger Count|
+-------------------+----------------------------+
|US                 |659837                      |
|Asia               |86398                       |
|Europe             |48136                       |
|Canada             |39798                       |
|Mexico             |29206                       |
|Middle East        |14769                       |
|Australia / Oceania|12973                       |
|Central America    |8970                        |
|South America      |3685                        |
+-------------------+----------------------------+



<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Convertir a csv
</div>

In [17]:
# Crear el path completo del archivo
output_file = os.path.join(output_path, "max_passenger_region.csv")

# Guardar como archivo CSV usando Pandas
max_passenger_region.toPandas()\
.to_csv(output_file, index=False)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Análisis descriptivo de las variables numéricas passenger count y adjusted passenger count
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Se observa una discrepancia entre las variables Passenger Count y Adjusted Passenger Count. Es probable que la columna Adjusted Passenger Count se refiera al listado final de pasajeros que abordaron el vuelo, incluyendo a usuarios que adquirieron el boleto a última hora en el  <i>"counter"</i> de la aerolínea. La diferencia entre ambas columnas es mínima pero importante, pues refleja la diferencia entre quienes posiblemente programaron sus vuelos con antelación y los que por alguna razón adquirieron el boleto a última hora.
</div>



In [18]:
# Calcular la media y la desvicación standard en las columnas passenger count y adjusted passenger count
pass_stats = df_1.agg(F.round(F.mean("Passenger Count"),2).alias("Mean Passenger Count"),
        (F.round(F.stddev("Passenger Count"),2). alias("Std Passenger Count")),
        (F.round(F.mean("Adjusted Passenger Count"),2).alias("Mean Adjusted Passenger Count")),
        (F.round(F.stddev("Adjusted Passenger Count"),2).alias("Std Adjusted Passenger Count")),
        )
# Mostrar el resultado
pass_stats.show()

+--------------------+-------------------+-----------------------------+----------------------------+
|Mean Passenger Count|Std Passenger Count|Mean Adjusted Passenger Count|Std Adjusted Passenger Count|
+--------------------+-------------------+-----------------------------+----------------------------+
|            29240.52|           58319.51|                     29331.92|                    58284.18|
+--------------------+-------------------+-----------------------------+----------------------------+



<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Analísis descriptivo por aerolínea de las variables de pasajeros en el período 2005-2016
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
American Airline tiene el mayor promedio de pasajeros y la stddev mas baja de todas las aerolíneas, lo que indica un tráfico de pasajeros estable y considerable.
</div>

In [19]:
# Calcular por aerolínea la media y la desviación standard en la columna Adjusted Passenger Count
airline_pass_stats = df_1.groupBy("Operating Airline").agg(
        F.round(F.mean("Adjusted Passenger Count"),2).alias("Mean Adjusted Passenger Count"),
        F.round(F.stddev("Adjusted Passenger Count"),2).alias ("Std Adjusted Passenger Count"),
        ).orderBy("Mean Adjusted Passenger Count", ascending=False)

# Mostrar el resultado
airline_pass_stats.show()

+--------------------+-----------------------------+----------------------------+
|   Operating Airline|Mean Adjusted Passenger Count|Std Adjusted Passenger Count|
+--------------------+-----------------------------+----------------------------+
|   American Airlines|                    127164.39|                    22044.24|
|  Southwest Airlines|                     81223.35|                    60310.95|
|      Virgin America|                     74405.35|                    68539.61|
|     United Airlines|                     72827.22|                   111353.45|
|     Delta Air Lines|                     68515.42|                    52420.02|
|          US Airways|                     55317.82|                    17368.96|
|United Airlines -...|                     49365.52|                   101159.05|
|    SkyWest Airlines|                     37083.88|                    47114.36|
|    JetBlue Airways |                     35261.14|                    15781.56|
|  Northwest Air

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El gráfico muestra las barras con el promedio de pasajeros por aerolínea junto a las líneas que representan la desviación estándar. Cuanto más largas son las líneas, mayor es la variabilidad en el número de pasajeros de la aerolínea. La línea más corta indica que el flujo de pasajeros es más consistente en sus vuelos. United Airlines, en sus dos entradas, es la compañía que muestra la mayor variabilidad en el número de pasajeros.
</div>

In [20]:
# Convertir a pandas 
airline_pass_stats_pd = airline_pass_stats.toPandas()

# Crear un gráfico de barras interactivo
fig = px.bar(airline_pass_stats_pd, 
             x="Operating Airline", 
             y="Mean Adjusted Passenger Count", 
             error_y="Std Adjusted Passenger Count",
             title="Promedio de Pasajeros por Aerolínea",
             labels={"Operating Airline": "Aerolíneas", "Mean Adjusted Passenger Count": "Promedio de Pasajeros"},
             )

# Mostrar gráfico
fig.show()



![](newplot1.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Calcular la media de pasajeros por año
</div>

In [21]:
year_pass_stats = df_1.groupBy("Year").agg(F.round(F.mean("Adjusted Passenger Count"),2)\
        .alias("Mean Adjusted Passenger Count"),
        F.round(F.stddev("Adjusted Passenger Count"),2)\
        .alias("Std Adjusted Passenger Count")
        ).orderBy("Year", ascending=True)

year_pass_stats.show()

+----+-----------------------------+----------------------------+
|Year|Mean Adjusted Passenger Count|Std Adjusted Passenger Count|
+----+-----------------------------+----------------------------+
|2005|                     25002.34|                    54707.74|
|2006|                     24529.88|                    53358.71|
|2007|                     25401.59|                    52626.75|
|2008|                     26100.87|                    50726.96|
|2009|                     26887.03|                    55063.06|
|2010|                     28482.45|                    57745.14|
|2011|                     29529.09|                    57905.05|
|2012|                     32276.64|                    58587.49|
|2013|                     33145.63|                    69484.16|
|2014|                      34470.1|                    63681.88|
|2015|                     34292.53|                    62731.75|
|2016|                     30811.85|                    56349.96|
+----+----

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El gráfico muestra el promedio de pasajeros ajustados por año. A partir del 2005, se observa un crecimiento moderado y constante hasta alcanzar su máximo en 2014. A partir de entonces comienza a disminuir. Las lineas verticales indican una alta variabilidad o dispersión en el número de pasajeros, siendo 2013 donde la stddev es mayor +-69484.16
</div>

In [22]:
# Convertir a Pandas 
year_pass_stats_pd = year_pass_stats.toPandas()

# Crear gráfico de líneas con barras de error
fig = px.line(year_pass_stats_pd, 
              x="Year", 
              y="Mean Adjusted Passenger Count", 
              error_y="Std Adjusted Passenger Count",
              title="Tendencia de Pasajeros Promedio por Año con Variabilidad",
              labels={"Year": "Año", "Mean Adjusted Passenger Count": "Promedio de Pasajeros"},
              )

# Mostrar gráfico
fig.show()

![](newplot2.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de pasajeros por destino y el porcentaje que representan en el período 2005-2016
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Mas de 3/4 de los pasajeros toman vuelos nacionales.
</div>

In [23]:
# Calcular el total de pasajeros
total_pass = df_1.agg(F.sum("Adjusted Passenger Count").alias("Total Passengers")).collect()[0][0]

# Agrupar los datos por 'GEO Summary' y calcular el total de pasajeros por categoría
pass_distribution = (
    df_1.groupBy("GEO Summary")
    .agg(F.sum("Adjusted Passenger Count").alias("Total Passengers"))
    .withColumn("Percentage", F.format_number((F.col("Total Passengers") / total_pass) * 100, 2))
    .orderBy("Total Passengers", ascending=False)
)

# Mostrar los resultados
pass_distribution.show()

+-------------+----------------+----------+
|  GEO Summary|Total Passengers|Percentage|
+-------------+----------------+----------+
|     Domestic|       339042637|     77.02|
|International|       101141443|     22.98|
+-------------+----------------+----------+



In [24]:

# Convertir a pandas
pass_distribution_pd = pass_distribution.toPandas()

# Crear el gráfico Sunburst
fig = px.sunburst(
    pass_distribution_pd,
    path=['GEO Summary'],  # Definir el camino (jerarquía)
    values='Total Passengers',  # Valores para el gráfico
    title="Distribución de Pasajeros por Destino (2005-2016)",
)

# Ajustar el tamaño del gráfico
fig.update_layout(width=600, height=400)

# Mostrar el gráfico
fig.show()

![](newplot3.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de pasajeros por región en el período 2005-2016
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Los vuelos internacionales movilizan el mayor numero de pasajeros hacia Asia, Europa y Canada.
</div>

In [25]:
# Agrupar por "GEO Region" y "GEO Summary" y calcular el total de pasajeros
total_pass_region_summary = df_1.groupBy("GEO Region", "GEO Summary") \
    .agg(F.sum("Adjusted Passenger Count").alias("Total Adjusted Passengers")) \
    .withColumn("Percentage", F.format_number((F.col("Total Adjusted Passengers") / total_pass) * 100, 2)) \
    .orderBy("Total Adjusted Passengers", ascending=False)

# Mostrar el resultado
total_pass_region_summary.show()

+-------------------+-------------+-------------------------+----------+
|         GEO Region|  GEO Summary|Total Adjusted Passengers|Percentage|
+-------------------+-------------+-------------------------+----------+
|                 US|     Domestic|                339042637|     77.02|
|               Asia|International|                 44213493|     10.04|
|             Europe|International|                 26695446|      6.06|
|             Canada|International|                 13901776|      3.16|
|             Mexico|International|                  8084752|      1.84|
|Australia / Oceania|International|                  4786892|      1.09|
|        Middle East|International|                  1852943|      0.42|
|    Central America|International|                  1355400|      0.31|
|      South America|International|                   250741|      0.06|
+-------------------+-------------+-------------------------+----------+



In [26]:
# Excluir la región 'US' y calcular el total de pasajeros de las otras 8 regiones
total_pass_region_summary_excl_us = total_pass_region_summary.filter(F.col("GEO Region") != "US")

# Calcular el total de pasajeros sin la región 'US'
total_pass_without_us = total_pass_region_summary_excl_us.agg(F.sum("Total Adjusted Passengers").alias("Total Adjusted Passengers Without US")).collect()[0][0]

# Calcular el porcentaje respecto al total de las otras 8 regiones
total_pass_region_summary_excl_us = total_pass_region_summary_excl_us.withColumn(
    "Percentage Excl US", F.format_number((F.col("Total Adjusted Passengers") / total_pass_without_us) * 100, 2)
)

# Mostrar el resultado
total_pass_region_summary_excl_us.show()


+-------------------+-------------+-------------------------+----------+------------------+
|         GEO Region|  GEO Summary|Total Adjusted Passengers|Percentage|Percentage Excl US|
+-------------------+-------------+-------------------------+----------+------------------+
|               Asia|International|                 44213493|     10.04|             43.71|
|             Europe|International|                 26695446|      6.06|             26.39|
|             Canada|International|                 13901776|      3.16|             13.74|
|             Mexico|International|                  8084752|      1.84|              7.99|
|Australia / Oceania|International|                  4786892|      1.09|              4.73|
|        Middle East|International|                  1852943|      0.42|              1.83|
|    Central America|International|                  1355400|      0.31|              1.34|
|      South America|International|                   250741|      0.06|        

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
En el gráfico se puede observar la marcada desproporción del flujo de pasajeros entre US y el resto de regiones. La primera se apróxima a los 440 mill, en tanto Asia, la segunda ruta con mayor tráfico alcanza casi los 45 mill.
</div>

In [27]:
# Convertir a Pandas 
total_pass_region_summary_pd = total_pass_region_summary.toPandas()

# Crear el gráfico de líneas
fig = px.line(
    total_pass_region_summary_pd,
    x="GEO Region",
    y="Total Adjusted Passengers",
    title="Total de Pasajeros por Región (2005-2016)",
    labels={"Total Adjusted Passengers": "Total de Pasajeros", "GEO Region": "Región"}
)

# Mostrar el gráfico
fig.show()

![](newplot4.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Promedio mensual de pasajeros por región para el período 2005-2016
</div>

In [28]:
# Agrupar por "GEO Region" y obtener el promedio de "Adjusted Passenger Count"
avg_pass_region = df_1.groupBy("GEO Region").agg(
    F.round(F.mean("Adjusted Passenger Count"),2).alias("Mean Adjusted Passenger Count")
).orderBy("Mean Adjusted Passenger Count", ascending=False)

# Mostrar el resultado
avg_pass_region.show(truncate=False)

+-------------------+-----------------------------+
|GEO Region         |Mean Adjusted Passenger Count|
+-------------------+-----------------------------+
|US                 |58485.88                     |
|Asia               |13508.55                     |
|Europe             |12779.06                     |
|Canada             |9803.79                      |
|Middle East        |8658.61                      |
|Mexico             |7250.9                       |
|Australia / Oceania|6495.1                       |
|Central America    |4946.72                      |
|South America      |2786.01                      |
+-------------------+-----------------------------+



<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de meses registrados por region en el período 2005-2016 
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Las rutas domésticas de US acumulan 5797 meses, en tanto South America solo 90 meses. Asia 3297.
</div>

In [29]:
# Contar el número de meses registrados por cada región
region_months_count = df_1.groupBy("GEO Region").agg(
    F.count("Adjusted Passenger Count").alias("Months Recorded")
).orderBy(["Months Recorded", "GEO Region"], ascending=[False, False])

# Mostrar cuántos meses están registrados por región
region_months_count.show(truncate=False)

+-------------------+---------------+
|GEO Region         |Months Recorded|
+-------------------+---------------+
|US                 |5797           |
|Asia               |3273           |
|Europe             |2089           |
|Canada             |1418           |
|Mexico             |1115           |
|Australia / Oceania|737            |
|Central America    |274            |
|Middle East        |214            |
|South America      |90             |
+-------------------+---------------+



<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de pasajeros por destino, región, terminal y boarding area 2005-2016
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El uso de los terminales y las áreas de abordaje muestra un patrón de uso variable. Algunas rutas internacionales (Asia, Canada, Mexico) usan las terminales de los vuelos nacionales en tanto algunas rutas domésticas movilizan a los pasajeros desde el aeropuerto internacional. Habría que profundizar si existe algun patrón definido para este uso. ¿La asignación del terminal y boarding area esta relacionado con el flujo de pasajeros de las aerolíneas?.
</div>

In [30]:
# Agrupar y sumar 'Adjusted Passenger Count', luego ordenar por 'GEO Region' y total de pasajeros
total_pass_region_terminal_boarding_pd = (
    df_1.groupBy("GEO Region", "GEO Summary", "Terminal", "Boarding Area")
    .agg(F.sum("Adjusted Passenger Count").alias("Total Adjusted Passengers"))
    .orderBy("GEO Region", F.col("Total Adjusted Passengers").desc())
)

# Mostrar el resultado
total_pass_region_terminal_boarding_pd.show(50,truncate=False)

+-------------------+-------------+-------------+-------------+-------------------------+
|GEO Region         |GEO Summary  |Terminal     |Boarding Area|Total Adjusted Passengers|
+-------------------+-------------+-------------+-------------+-------------------------+
|Asia               |International|International|G            |28125128                 |
|Asia               |International|International|A            |16087649                 |
|Asia               |International|Terminal 3   |F            |712                      |
|Asia               |International|Other        |Other        |4                        |
|Australia / Oceania|International|International|G            |4118069                  |
|Australia / Oceania|International|International|A            |668823                   |
|Canada             |International|Terminal 3   |F            |5667272                  |
|Canada             |International|Terminal 3   |E            |2902198                  |
|Canada   

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Principales 5 aerolíneas por Región 2005-2016
</div>

In [31]:
# Agrupar por 'GEO Region' y 'Operating Airline', calcular el total de pasajeros y asignar ranking
ranked_airlines_region = (
    df_1.groupBy("GEO Region", "Operating Airline")
    .agg(F.sum("Adjusted Passenger Count").alias("Total Passengers"))
    .withColumn("Rank", F.row_number().over(Window.partitionBy("GEO Region").orderBy(F.col("Total Passengers").desc())))
    .filter(F.col("Rank") <= 5)  # Filtrar las 5 principales aerolíneas por región
    .orderBy("GEO Region", "Rank")
)

# Mostrar los resultados
ranked_airlines_region.show(50, truncate=False)

+-------------------+--------------------------------+----------------+----+
|GEO Region         |Operating Airline               |Total Passengers|Rank|
+-------------------+--------------------------------+----------------+----+
|Asia               |United Airlines - Pre 07/01/2013|13053369        |1   |
|Asia               |United Airlines                 |4499526         |2   |
|Asia               |Cathay Pacific                  |4417302         |3   |
|Asia               |Singapore Airlines              |3804635         |4   |
|Asia               |EVA Airways                     |3384020         |5   |
|Australia / Oceania|Air New Zealand                 |1930156         |1   |
|Australia / Oceania|United Airlines - Pre 07/01/2013|1691485         |2   |
|Australia / Oceania|Qantas Airways                  |668823          |3   |
|Australia / Oceania|United Airlines                 |496428          |4   |
|Canada             |Air Canada                      |6680071         |1   |

In [32]:
# Filtrar solo los datos de United Airlines y agrupar por GEO Summary
united_passengers_by_geo = (
    df_1.filter(F.col("Operating Airline")\
    .isin("United Airlines", "United Airlines - Pre 07/01/2013"))
    .groupBy("GEO Summary")
    .agg(F.sum("Adjusted Passenger Count").alias("Total Passengers", ))
)

# Calcular el total general de pasajeros para United Airlines
total_passengers = united_passengers_by_geo.agg(F.sum("Total Passengers")\
    .alias("Total Overall")).collect()[0]["Total Overall"]

# Agregar la columna del porcentaje
united_passengers_by_geo = united_passengers_by_geo.withColumn(
    "Percentage of Total", F.col("Total Passengers") / total_passengers * 100
)

# Mostrar los resultados
united_passengers_by_geo.show(truncate=False)


+-------------+----------------+-------------------+
|GEO Summary  |Total Passengers|Percentage of Total|
+-------------+----------------+-------------------+
|International|33849703        |19.761033821828626 |
|Domestic     |137445500       |80.23896617817138  |
+-------------+----------------+-------------------+



<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
United Airlines es la principal operadora, por el número de pasajeros movilizados, en US, Asia, Europa y Mexico y con una cuota importante de mercado en Australia/Oceania y Canada. 
</div>

In [33]:
# Convertir a Pandas DataFrame
ranked_airlines_by_region_pd = ranked_airlines_region.toPandas()

# Añadir un "jitter" al eje Y para separar las burbujas
ranked_airlines_by_region_pd["Y_Jitter"]\
    = ranked_airlines_by_region_pd["Total Passengers"] + np.random.uniform(-500000, 500000, size=len(ranked_airlines_by_region_pd))

# Crear gráfico de dispersión
fig = px.scatter(
    ranked_airlines_by_region_pd,
    x="GEO Region",
    y="Y_Jitter",
    size="Total Passengers",
    color="Operating Airline",
    title="Primeras 5 aerolíneas por Región 2005-2016",
    size_max=60,
    #template="plotly_white"
).update_traces(textposition="top center").update_layout(
    xaxis_title="Región",
    yaxis_title="Total de Pasajeros",
)

# Mostrar gráfico
fig.show()

![](newplot5.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de pasajeros por aerolíneas en el período 2005-2016
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
United Airlines tiene dos entradas en el dataframe. United Airlines - Pre 07/01/2013 cubre el período 2005-2013 y United Airlines, el período 2005-2016. Esto dificulta el análisis pues no está clara la razón. Lo cierto es que United Airlines - Pre 07/01/2013, muestra una cifra superior de pasajeros a pesar que los datos llegan hasta 2013. Esta situación se observa tanto en vuelos nacionales como internacionales. Habría que profundizar más sobre este punto. Si se consolidan ambas cifras, sin que ello implique una duplicación de datos, la movilización de pasajeros por United Airlines convertiría a ésta aerolínea, en comparación con las otras, en la principal compañía que opera desde el Aeropuerto de San Francisco en líneas nacionales e internacionales.
</div>

In [34]:
def summarize_passengers_by_airline(df_1, geo_summary):
    return (
        df_1.filter(df_1["GEO Summary"] == geo_summary)
        .groupBy("Operating Airline")
        .agg(
            F.sum("Adjusted Passenger Count").alias("Total Passengers"),
            F.concat_ws("-", F.min("Year").cast("string"),\
            F.max("Year").cast("string")).alias("Period")
        )
        .orderBy("Total Passengers", ascending=False)
    )

In [35]:
domestic_summary = summarize_passengers_by_airline(df_1, "Domestic")
domestic_summary.show(77, truncate=False)


+--------------------------------+----------------+---------+
|Operating Airline               |Total Passengers|Period   |
+--------------------------------+----------------+---------+
|United Airlines - Pre 07/01/2013|81757535        |2005-2013|
|United Airlines                 |55687965        |2005-2016|
|American Airlines               |34588714        |2005-2016|
|SkyWest Airlines                |33068349        |2005-2016|
|Virgin America                  |26350566        |2007-2016|
|Delta Air Lines                 |25948711        |2005-2016|
|Southwest Airlines              |25098015        |2007-2016|
|US Airways                      |16816616        |2005-2015|
|Alaska Airlines                 |10614399        |2005-2016|
|JetBlue Airways                 |7827973         |2007-2016|
|Northwest Airlines              |5571581         |2005-2010|
|Frontier Airlines               |4624796         |2005-2016|
|AirTran Airways                 |2388648         |2005-2014|
|Hawaiia

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de pasajeros por aerolínea en vuelos internacionales en el período 2005-2016
</div>

In [36]:
international_summary = summarize_passengers_by_airline(df_1, "International")
international_summary.show(77, truncate=False)

+--------------------------------+----------------+---------+
|Operating Airline               |Total Passengers|Period   |
+--------------------------------+----------------+---------+
|United Airlines - Pre 07/01/2013|24575788        |2005-2013|
|United Airlines                 |9273915         |2013-2016|
|Air Canada                      |6680071         |2005-2016|
|Lufthansa German Airlines       |4979907         |2005-2016|
|British Airways                 |4547282         |2005-2016|
|Cathay Pacific                  |4417302         |2005-2016|
|Singapore Airlines              |3804635         |2005-2016|
|EVA Airways                     |3384020         |2005-2016|
|Air France                      |2989982         |2005-2016|
|Philippine Airlines             |2644148         |2005-2016|
|SkyWest Airlines                |2643424         |2005-2016|
|Alaska Airlines                 |2576674         |2005-2016|
|China Airlines                  |2543239         |2005-2016|
|Virgin 

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total anual de pasajeros por destino
</div>

In [37]:
# Agrupar por Año y Región y sumar el total de pasajeros
annual_pass_summary = df_1.groupBy(
    "Year",                # Agrupar por Año
    "GEO Summary"           # Agrupar por Destino
).agg(
    F.sum("Adjusted Passenger Count").alias("Total Passengers")  # Sumar el total de pasajeros
).orderBy("GEO Summary", "Year")

# Mostrar el resultado
annual_pass_summary.show(30, truncate=False)

+----+-------------+----------------+
|Year|GEO Summary  |Total Passengers|
+----+-------------+----------------+
|2005|Domestic     |13104950        |
|2006|Domestic     |24896096        |
|2007|Domestic     |26611925        |
|2008|Domestic     |28300113        |
|2009|Domestic     |29055818        |
|2010|Domestic     |30446208        |
|2011|Domestic     |31947168        |
|2012|Domestic     |34897218        |
|2013|Domestic     |35266950        |
|2014|Domestic     |36885221        |
|2015|Domestic     |38823001        |
|2016|Domestic     |8807969         |
|2005|International|4271679         |
|2006|International|8685316         |
|2007|International|9178909         |
|2008|International|9102428         |
|2009|International|8397816         |
|2010|International|8945026         |
|2011|International|9098263         |
|2012|International|9579991         |
|2013|International|9744814         |
|2014|International|10269879        |
|2015|International|11244093        |
|2016|Intern

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Calcular el crecimiento interanual de pasajeros por destino.

Se aprecia una importante caída del número de pasajeros internacionales en 2009, probablemente relacionado al hecho de la crisis financiera. En vuelos nacionales el crecimiento es de 2.6 puntos porcentuales, en comparación con el año previo de 6.3%.
</div>

In [38]:
# Agrupar por Año y Región y sumar el total de pasajeros
annual_pass_summary = df_1.groupBy(
    "Year",                # Agrupar por Año
    "GEO Summary"          # Agrupar por Región
).agg(
    F.sum("Adjusted Passenger Count").alias("Total Passengers")  # Sumar el total de pasajeros
).orderBy("GEO Summary", "Year")

# Crear una ventana para obtener el total de pasajeros del año anterior
window_spec = Window.partitionBy("GEO Summary").orderBy("Year")

# Calcular el crecimiento anual en porcentaje
annual_pass_summary_with_growth = annual_pass_summary.withColumn(
    "Previous Year Passengers", F.lag("Total Passengers").over(window_spec)
).withColumn(
    "Growth Percentage", 
    (F.col("Total Passengers") - F.col("Previous Year Passengers")) / F.col("Previous Year Passengers") * 100
)

# Redondear el porcentaje de crecimiento a dos decimales
annual_pass_summary_with_growth = annual_pass_summary_with_growth.withColumn(
    "Growth Percentage", F.format_number("Growth Percentage", 2)
)

# Filtrar los años entre 2006 y 2015 (excluyendo 2005 y 2016)
annual_pass_summary_with_growth_filtered = annual_pass_summary_with_growth.filter(
    (F.col("Year") >= 2006) & (F.col("Year") <= 2015)
)

# Mostrar el resultado
annual_pass_summary_with_growth.show(30, truncate=False)


+----+-------------+----------------+------------------------+-----------------+
|Year|GEO Summary  |Total Passengers|Previous Year Passengers|Growth Percentage|
+----+-------------+----------------+------------------------+-----------------+
|2005|International|4271679         |null                    |null             |
|2006|International|8685316         |4271679                 |103.32           |
|2007|International|9178909         |8685316                 |5.68             |
|2008|International|9102428         |9178909                 |-0.83            |
|2009|International|8397816         |9102428                 |-7.74            |
|2010|International|8945026         |8397816                 |6.52             |
|2011|International|9098263         |8945026                 |1.71             |
|2012|International|9579991         |9098263                 |5.29             |
|2013|International|9744814         |9579991                 |1.72             |
|2014|International|10269879

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Calcular el crecimiento interanual de pasajeros.
</div>

In [39]:
# Agrupar por Año y sumar el total de pasajeros
annual_pass_summary = df_1.groupBy(
    "Year"                # Agrupar solo por Año
).agg(
    F.sum("Adjusted Passenger Count").alias("Total Passengers")  # Sumar el total de pasajeros
).orderBy("Year")

# Crear una ventana para obtener el total de pasajeros del año anterior
window_spec = Window.orderBy("Year")

# Calcular el crecimiento interanual en porcentaje
annual_pass_summary_with_growth = annual_pass_summary.withColumn(
    "Previous Year Passengers", F.lag("Total Passengers").over(window_spec)
).withColumn(
    "Growth Percentage", 
    (F.col("Total Passengers") - F.col("Previous Year Passengers")) / F.col("Previous Year Passengers") * 100
)

# Redondear el porcentaje de crecimiento a dos decimales
annual_pass_summary_with_growth = annual_pass_summary_with_growth.withColumn(
    "Growth Percentage", F.format_number("Growth Percentage", 2)
)

# Filtrar los años entre 2006 y 2015 (excluyendo 2005 y 2016)
annual_pass_summary_with_growth_filtered = annual_pass_summary_with_growth.filter(
    (F.col("Year") >= 2006) & (F.col("Year") <= 2015)
)

# Mostrar el resultado
annual_pass_summary_with_growth_filtered.show(30, truncate=False)


+----+----------------+------------------------+-----------------+
|Year|Total Passengers|Previous Year Passengers|Growth Percentage|
+----+----------------+------------------------+-----------------+
|2006|33581412        |17376629                |93.26            |
|2007|35790834        |33581412                |6.58             |
|2008|37402541        |35790834                |4.50             |
|2009|37453634        |37402541                |0.14             |
|2010|39391234        |37453634                |5.17             |
|2011|41045431        |39391234                |4.20             |
|2012|44477209        |41045431                |8.36             |
|2013|45011764        |44477209                |1.20             |
|2014|47155100        |45011764                |4.76             |
|2015|50067094        |47155100                |6.18             |
+----+----------------+------------------------+-----------------+



<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Generación de gráfico anual de pasajeros por región.
</div>

In [40]:
# Función para analizar el total de pasajeros por region y año
def create_annual_passenger_chart(df_1, region_col, exclude_region=None, title=None, color_label="Región"):
    if exclude_region:
        df_1 = df_1.filter(df_1[region_col] != exclude_region)
    
    # Agrupar y sumar
    grouped_df_1 = (
        df_1.groupBy("Year", region_col)
        .agg(F.sum("Adjusted Passenger Count").alias("Total Passengers"))
        .orderBy("Year", region_col)
    )
    
    # Convertir a Pandas y graficar
    fig = px.line(
        grouped_df_1.toPandas(),
        x="Year",
        y="Total Passengers",
        color=region_col,
        title=title,
        labels={"Year": "Año", "Total Passengers": "Total de Pasajeros", region_col: color_label},
        template='plotly_white'
    )
    fig.update_layout(
        legend=dict(title=color_label, font=dict(size=13), orientation='v', x=1.05, y=1),
        margin=dict(r=150)
    )
    return grouped_df_1, fig

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total anual de pasajeros por región.
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El año inicial y final del dataframe muestran datos parciales. 2005 inicia en julio y 2016 culmina en marzo. Esto explica tanto el ascenso como la caída abrupta que se observa en el gráfico.
Igualmente, se nota una tendencia creciente en el flujo de pasajeros en la ruta nacional desde el 2006 hasta el 2015, y un movimiento estable en la ruta internacional en el mismo período.
</div>

In [41]:
# Distribución anual de pasajeros por vuelos
df_summary, fig = create_annual_passenger_chart(df_1, "GEO Summary", title="Total Anual de Pasajeros por Destino")
fig.show()

![](newplot6.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
En las rutas internacionales, las líneas de cada una de las regiones muestran poca variabilidad a lo largo del período, a excepción de Asia y Europa, y en menor medida Canada y Mexico. En el caso de South America sólo hubo operaciones en el período 2010-2014. LAN Peru fue la única aerolínea que cubrió la ruta. El cese de operaciones posiblemente estuvo asociado al bajo flujo de pasajeros y al costo del boleto. Habría que profundizar mas al respecto.
</div>

In [42]:
# Dsitribución anual de pasajeros por regiones excluída US 
df_summary, fig = create_annual_passenger_chart(df_1, "GEO Region", exclude_region="US", title="Total Anual de Pasajeros por Región sin EEUU")
fig.show()

![](newplot7.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Generación de gráfico para el análisis de vuelos.
</div>

In [43]:
# Función para analizar los vuelos en relación con Geo summary y agrupar por aerolíneas y año
def analyze_flights(df_1, geo_summary, title):
    filtered_df_1 = df_1.filter(df_1["GEO Summary"] == geo_summary)
    result = filtered_df_1.groupBy("Operating Airline", "Year").agg(
        F.sum("Adjusted Passenger Count").alias("Total Passengers")
    ).orderBy("Operating Airline", "Year")
    #result.show() # Mostrar el resultado
    fig = px.scatter(
        result.toPandas(),
        x="Year",
        y="Operating Airline",
        size="Total Passengers",
        color="Operating Airline",
        title=title
    )
    fig.show()

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total anual de pasajeros por aerolínea en vuelos nacionales
</div>

In [44]:
# Pasajeros por aerolíneas en vuelos nacionales
analyze_flights(df_1, "Domestic", "Total Anual de Pasajeros por Aerolínea (Nacional)")


![](newplot8.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total anual de pasajeros por aerolínea en vuelos internacionales
</div>

In [45]:
# Pasajeros por aerolíneas en vuelos internacionales 
analyze_flights(df_1, "International", "Total Anual Pasajeros por Aerolínea (Internacional)")

![](newplot9.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Generación de gráfico para el análisis de vuelos con tarifas reducidas.
</div>

In [46]:
# Función que analiza los datos de los pasajeros por año y aerolínea en función del precio 
def analyze_fare_summary(df_1, geo_summary, price_category, title):
    # Filtrar los datos
    filtered_df_1 = df_1.filter(
        (df_1["GEO Summary"] == geo_summary) &
        (df_1["Price Category Code"] == price_category)
    )
    
    # Calcular totales por aerolínea
    summary = filtered_df_1.groupBy("Operating Airline", "Year").agg(
        F.sum("Adjusted Passenger Count").alias("Total Passengers")
    )
    
    # Calcular totales anuales
    yearly_totals = filtered_df_1.groupBy("Year").agg(
        F.sum("Adjusted Passenger Count").alias("Total Yearly Passengers")
    )
    
    # Combinar y calcular porcentajes
    result = summary.join(yearly_totals, on="Year", how="left").withColumn(
        "% Passenger", F.format_number(
            (F.col("Total Passengers") / F.col("Total Yearly Passengers")) * 100, 2
        )
    ).orderBy("Year", "Total Passengers", ascending=False)
    #result.show() # Mostrar el resultado
    
    # Convertir a Pandas para visualización
    pandas_df = result.toPandas()
    
    # Crear gráfico
    fig = px.bar(
        pandas_df,
        x="Year",
        y="Total Passengers",
        color="Operating Airline",
        text="% Passenger",
        title=title,
        labels={
            "Total Passengers": "Total de Pasajeros",
            "Operating Airline": "Aerolínea",
            "Year": "Año"
        }
    )
    fig.show()


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Distribución Anual de Pasajeros por Aerolíneas en vuelos nacionales con Tarifas Reducidas
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
A partir del 2008, se observa un crecimiento sostenido de la oferta de tarifas reducidas en las rutas domésticas de Virgin America, en competencia directa con Southwest Airlines. 

En los vuelos internacionales, a partir del 2010, parece que Virgin America introduce la tarifa reducida, pues antes de ese año no existen registros de otra aerolínea con tal oferta. Incluso, a partir del año siguiente otros operadores intentaron competir en ese segmento tarifario pero sin mayor éxito. La data no muestra registros sostenidos en el tiempo para otras aerolíneas distintas a Virgin America. Tal vez la compañía hizo de la tarifa reducida un distintivo de marca. Habría que profundizar en el tema para verificar lo que muestra la data.
</div>

In [47]:
# Análisis para vuelos domésticos
analyze_fare_summary(df_1, "Domestic", "Low Fare", 
    "Distribución Anual de Pasajeros por Aerolíneas Nacionales con Tarifas Reducidas")


![](newplot10.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Distribución Anual de Pasajeros por Aerolíneas en vuelos internacionales con Tarifas Reducidas
</div>

In [48]:
# Análisis para vuelos internacionales
analyze_fare_summary(df_1, "International", "Low Fare", 
    "Distribución Anual de Pasajeros por Aerolíneas Internacionales con Tarifas Reducidas")


![](newplot11.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total mensual de pasajeros y su distribución por destino
</div>

In [49]:
# Calcular el total de pasajeros por mes en Domestic e International vuelos
monthly_summary = df_1.groupBy("Month").agg(
    F.sum("Adjusted Passenger Count").alias("Total Passengers"),
    F.sum(F.when(df_1["GEO Summary"] == "Domestic", df_1["Adjusted Passenger Count"]).otherwise(0)).alias("Domestic Passengers"),
    F.sum(F.when(df_1["GEO Summary"] == "International", df_1["Adjusted Passenger Count"]).otherwise(0)).alias("International Passengers")
)

# Añadir un índice de mes para ordenar correctamente
monthly_summary = monthly_summary.withColumn(
    "Month_Index", 
    F.when(F.col("Month") == "January", 1)
     .when(F.col("Month") == "February", 2)
     .when(F.col("Month") == "March", 3)
     .when(F.col("Month") == "April", 4)
     .when(F.col("Month") == "May", 5)
     .when(F.col("Month") == "June", 6)
     .when(F.col("Month") == "July", 7)
     .when(F.col("Month") == "August", 8)
     .when(F.col("Month") == "September", 9)
     .when(F.col("Month") == "October", 10)
     .when(F.col("Month") == "November", 11)
     .when(F.col("Month") == "December", 12)
)

# Ordenar por el índice de mes y seleccionar las columnas necesarias
monthly_summary = monthly_summary.orderBy("Month_Index").select(
    "Month", "Total Passengers", "Domestic Passengers", "International Passengers"
)

# Mostrar el resultado
#monthly_summary.show(12, truncate=False)


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Los meses con mayor afluencia de pasajeros son julio y agosto, coincidiendo probablemente con el período de vacaciones escolares. La misma tendencia se observa tanto en rutas domésticas como internacionales.
</div>

In [50]:
# Convertir el resultado a un DataFrame de Pandas y crear el gráfico de barras agrupadas
fig = px.bar(
    monthly_summary.toPandas(),  # Convertir directamente a Pandas dentro de la llamada
    x="Month",
    y=["Total Passengers", "Domestic Passengers", "International Passengers"],
    title="Distribución Mensual de Pasajeros (2005-2016)",
    labels={"value": "Total de Pasajeros", "variable": "Categoría", "Month": "Mes"},
    category_orders={"Month": ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]},
    text_auto=True
)

# Personalizar el diseño y mostrar el gráfico
fig.update_layout(
    barmode='group',
    xaxis_title="Mes",
    yaxis_title="Total de Pasajeros",
    template="plotly_white",
    legend_title="Categoría"
).show()


![](newplot12.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de actividad.
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Hay que extender la investigación más allá de la data actual para comprender a que se refieren estos datos, pues su definición ayuda poco a su comprensión. Es claro que no se refiere al total de vuelos, pues al hacer una división entre el total de pasajeros y esta columna los resultados son irreales. Mas de 400 mill de pasajeros no pueden ser transportados en 15007 vuelos.
</div>

In [51]:
# Contar el total de vuelos por 'Adjusted Activity Type Code'
activity_type_counts = df_1.groupBy("Adjusted Activity Type Code").count()

# Calcular el total de vuelos y crear una fila adicional
total_count = activity_type_counts.agg(F.sum("count").alias("count")).withColumn("Adjusted Activity Type Code", F.lit("Total"))

# Unir la tabla original con la fila del total
final_result = activity_type_counts.unionByName(total_count)

# Mostrar el resultado ordenado por 'count' excepto para "Total"
#final_result.orderBy(F.when(F.col("Adjusted Activity Type Code") == "Total", 1).otherwise(0), F.col("count").desc()).show()


In [52]:
# Convertir el resultado a pandas, ordenar y excluir la fila "Total"
final_result_pd = final_result.toPandas().sort_values(by=["Adjusted Activity Type Code", "count"], ascending=[True, False])
final_result_pd = final_result_pd[final_result_pd["Adjusted Activity Type Code"] != "Total"]

# Crear el gráfico con Plotly
fig = px.bar(
    final_result_pd,
    x="Adjusted Activity Type Code",
    y="count",
    title="Total del Tipo de Actividad",
    labels={"Adjusted Activity Type Code": "Tipo de Actividad", "count": "Total de Vuelos"},
    color="Adjusted Activity Type Code",  # Colorear las barras por tipo de actividad
    text_auto=True  # Mostrar los valores sobre las barras
)

# Ajustar el diseño y tamaño del gráfico
fig.update_layout(
    xaxis_title="Tipo de Actividad",
    yaxis_title="Total de Actividad",
    template="plotly_white",  # Estilo blanco
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    showlegend=False,  # No mostrar la leyenda
    width=600, height=400  # Ajustar el tamaño
)

# Mostrar el gráfico
fig.show()


![](newplot13.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Análisis correlativo
</div>

In [53]:
# Convertir las columnas categóricas a índices numéricos
categorical_columns = [
    'Operating Airline', 'Published Airline', 'GEO Summary', 'GEO Region', 
    'Price Category Code', 'Terminal', 'Boarding Area', 
    'Adjusted Activity Type Code', 'Month'
]

In [54]:
# Crear los indexadores para las columnas categóricas
indexers = [StringIndexer(inputCol=col, outputCol=col + "_Index") for col in categorical_columns]


In [55]:
# Aplicar el StringIndexer para cada columna categórica
df_transformed = df_1
for indexer in indexers:
    df_transformed = indexer.fit(df_transformed).transform(df_transformed)


In [56]:
# Seleccionar las columnas numéricas para la correlación
numerical_columns = [
    'Activity Period', 'Operating Airline_Index', 'Published Airline_Index',
    'GEO Summary_Index', 'GEO Region_Index','Adjusted Passenger Count', 
    'Price Category Code_Index', 'Terminal_Index', 'Boarding Area_Index',
    'Passenger Count', 'Adjusted Activity Type Code_Index', 'Year', 'Month_Index'
]

In [57]:
# Crear un ensamblador de características para combinar las columnas numéricas seleccionadas en una sola columna de características
assembler = VectorAssembler(inputCols=numerical_columns, outputCol="features")
df_transformed = assembler.transform(df_transformed)


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">



**Correlación positiva**. 

Se observa una fuerte correlación positiva entre Activity Period y Year, Operating Airline y Published Airline, Terminal y Boarding Area, Passenger Count y Adjusted Passenger Count. Ello corrobora lo observado en el análisis inicial del dataframe, en la que algunas columnas muestran los datos ajustados de columnas similares.
Geo Summary tiene una relación positiva media con Adjusted Passenger Count, Price Category Code, Terminal, Boarding Area Passenger Count.
Ajusted Passenger Count tiene relación positiva media con Geo Summary y Terminal, Boarding Area. 
Terminal y Boarding Area tienen una relación positiva media con Geo Summary, Adjusted Passenger Count, Passenger Count y muy débil con Price Category Code. 

**Correlación negativa**. 

Geo Summary y Geo Region muestran una relación negativa media entre ellas.
Geo Region muestra una débil relación negativa con Adjusted Passenger Count, Price Category Code, Terminal, Boarding Area y Boarding Area. Esto puede deverse a distintas razones, incluida la organización de los datos (agrupación por regiones y no por destinos específicos de cada aerolínea) y al uso del Terminal de San Francisco. Algunas aerolíneas con vuelos internacionales (Mexico, Canada) usan terminales y boarding area de vuelos nacionales.
Adjusted Passenger tiene una débil relación con Operating Airline, Published Airline y Geo Region.
Operating Airline y Published Airline tienen una relación negatova débil con Geo Summary, Adjusted Passenger Count, Terminal,Boarding Area, Passenger Count y Adjusted Activity Code.

**Correlación cero o cercana a cero**. 

La columna Month muestra 0 relación con Geo Summary, Geo Region, Terminal, Boarding Area, Adjusted Type Code y cercana a 0 con el resto de variables.
</div>

In [58]:
# Calcular la matriz de correlación y convertirla a DataFrame
correlation_matrix = Correlation.corr(df_transformed, "features").head()[0].toArray()
correlation_df = pd.DataFrame(correlation_matrix, columns=numerical_columns, index=numerical_columns)

# Formatear los valores de correlación a 2 decimales
correlation_df = correlation_df.applymap(lambda x: f"{x:.2f}")

# Crear el mapa de calor usando plotly.express
fig = px.imshow(
    correlation_df, 
    labels={'x': 'Características', 'y': 'Características', 'color': 'Correlación'},
    color_continuous_scale='RdBu', 
    zmin=-1, zmax=1,  # Ajustar el rango de colores entre -1 y 1
    title="Matriz de Correlación",
    text_auto=True  # Añadir los valores de correlación dentro de cada celda
)

# Ajustar el tamaño de la figura y rotar las etiquetas para mejorar la visualización
fig.update_layout(
    width=900,  # Ajusta el ancho de la figura
    height=800,  # Ajusta la altura de la figura
    xaxis_tickangle=-45,  # Rota las etiquetas del eje X
    yaxis_tickangle=0  # Rota las etiquetas del eje Y
)

# Mostrar el gráfico
fig.show()


![](newplot14.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Transformación y agregación de datos para el análisis
</div>


In [59]:
# Convertir "Activity Period" a formato de fecha
df_1 = df_1.withColumn("Activity Period", F.to_date(F.col("Activity Period").cast("string"), "yyyyMM"))

# Convertir "Year" a formato de fecha
df_1 = df_1.withColumn("Year", F.to_date(F.col("Year").cast("string"), "yyyy"))

# Renombrar las columnas
df_1 = df_1.withColumnRenamed("Activity Period", "Date").withColumnRenamed("Adjusted Passenger Count", "passengers")

#df_1.dtypes


In [60]:
# Obtener la lista de columnas actuales
columnas = df_1.columns

# Eliminar las columnas 'Adjusted Activity Type Code' y 'passengers'
columnas = [col for col in columnas if col not in ['Adjusted Activity Type Code', 'passengers']]

# Agrupar por las columnas restantes y sumar los valores
df_1 = df_1.groupBy(columnas).agg(F.sum("passengers").alias("passengers"))

#df_1.dtypes


In [61]:
# Crear una nueva columna 'month' extrayendo el mes de la columna 'Date'
df_1 = df_1.withColumn("month", F.month("Date"))

# Obtener los años únicos de la columna 'Date'
year_ = df_1.select(F.year("Date").alias("year")).distinct().orderBy("year").rdd.map(lambda row: row["year"]).collect()

# Obtener los meses únicos de la columna 'month'
month_ = df_1.select("month").distinct().orderBy("month").rdd.map(lambda row: row["month"]).collect()

# Obtener las aerolíneas únicas de la columna 'Operating Airline'
airline = df_1.select("Operating Airline").distinct().orderBy("Operating Airline").rdd.map(lambda row: row["Operating Airline"]).collect()

# Obtener las regiones únicas de la columna 'GEO Region'
region = df_1.select("GEO Region").distinct().orderBy("GEO Region").rdd.map(lambda row: row["GEO Region"]).collect()

#df_1.dtypes

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Pasajeros por Mes con Opciones de Filtrado.
</div>

In [62]:
# Función para procesar los datos
def process_passengers_month(df_1, columna, seleccion=['todo'], columna_data='passengers'):
    """
    Procesa los datos de pasajeros por mes utilizando PySpark.

    Parámetros:
    - df_1: DataFrame de PySpark.
    - columna: Columna para filtrar los datos (ejemplo: 'Operating Airline' o 'GEO Region').
    - seleccion: Lista de valores para filtrar en la columna. Por defecto, incluye todos los valores.
    - columna_data: Nombre de la columna que contiene los datos de pasajeros. Por defecto, 'passengers'.

    Retorna:
    - df_3: DataFrame de PySpark con los datos agregados y ordenados.
    """
    # Filtrar los datos
    df_2 = df_1 if seleccion == ['todo'] else df_1.filter(F.col(columna).isin(seleccion))

    # Agregar datos por fecha y calcular totales
    group_columns = ['Date'] if seleccion == ['todo'] else ['Date', columna]
    df_3 = df_2.groupBy(*group_columns).agg(F.sum(columna_data).alias(columna_data)).orderBy('Date')

    # DataFrame con los datos agregados y ordenados por fecha.
    return df_3

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
Calculamos la media movil para suavizar las fluctuaciones de datos y permitir una visualización más clara de las tendencias a largo plazo. Al calcular un promedio de los valores de pasajeros durante un período de tiempo específico (en este caso, con un rolling_period de 12 meses), la media móvil ayuda a reducir el ruido de datos mensuales y hace más evidente la dirección general del flujo de pasajeros. Esto es útil para identificar patrones estacionales o cambios significativos en los datos sin que las variaciones mensuales afecten la interpretación.

El uso de la media móvil también facilita la comparación de tendencias a través de diferentes categorías, como aerolíneas o regiones, haciendo que el análisis visual sea más accesible y efectivo. En términos de visualización, el gráfico resultante no solo muestra los datos reales, sino también una "línea suave" que refleja mejor la tendencia general.
</div>

In [63]:
# Función para graficar con opciones de agrupación, filtrado y media móvil
def plot_passengers_month(df_3, columna, seleccion=['todo'], columna_data='passengers', ancho=25, alto=15, 
                          roll_mean='N', rolling_period=12):
    # Identificar las columnas relacionadas para construir el título
    columna1, columna2 = 'Operating Airline', 'GEO Region'
    otra_columna = columna2 if columna == columna1 else columna1

    # Convertir los datos a Pandas para graficar
    datos_grafico = []
    for elemento_in_seleccion in (seleccion if seleccion != ['todo'] else ['todo']):
        # Filtrar los datos específicos para cada selección
        df_4 = df_3 if elemento_in_seleccion == 'todo' else df_3.filter(F.col(columna) == elemento_in_seleccion)
        df_4_pandas = df_4.toPandas()

        # Agregar los datos base al gráfico
        datos_grafico.append(
            pd.DataFrame({
                'Date': df_4_pandas['Date'],
                'Value': df_4_pandas[columna_data],
                'Group': elemento_in_seleccion
            })
        )

        # Calcular y agregar la media móvil si es necesario
        if roll_mean == 'Y':
            rolling_mean = df_4_pandas[columna_data].rolling(window=rolling_period, center=True).mean()
            datos_grafico.append(
                pd.DataFrame({
                    'Date': df_4_pandas['Date'],
                    'Value': rolling_mean,
                    'Group': f"{elemento_in_seleccion} rolling {rolling_period}"
                })
            )

    # Combinar todos los datos en un solo DataFrame de Pandas
    df_final = pd.concat(datos_grafico)

    # Generar el título del gráfico
    titulo = f"Pasajeros por mes, {', '.join(map(str, seleccion))} {columna}(s) y todas las {otra_columna}(s)"
    if len(titulo) > 78:
        titulo = titulo[:78] + '<br>' + titulo[78:]

    # Crear el gráfico utilizando Plotly Express
    fig = px.line(
        df_final,
        x='Date',
        y='Value',
        color='Group',
        title=titulo,
        labels={'Value': 'Pasajeros', 'Date': 'Mes'},
        width=ancho * 50,
        height=alto * 50
    )

    # Ajustar el diseño del gráfico
    fig.update_layout(
        plot_bgcolor='#F9FAFF',
        title_font_size=20,
        xaxis_title_font_size=15,
        yaxis_title_font_size=15,
        title_x=0.5
    )

    # Mostrar el gráfico
    fig.show()


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Pasajeros por mes, todas las regiones y aerolíneas.
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El gráfico muestra una clara tendencia de crecimiento y estacionalidad.
</div>

In [64]:
# Comportamiento del flujo de pasajeros por año durante 2005-2016
df_3_all = process_passengers_month(df_1, "GEO Region")
plot_passengers_month(df_3_all, "GEO Region")


![](newplot15.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Tendencia mensual por regiones.
</div>

In [65]:
# Comportamiento anual del flujo de pasajeros por regiones y mes
df_3_region = process_passengers_month(df_1, "GEO Region", region)
plot_passengers_month(df_3_region, "GEO Region", region)


![](newplot16.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de pasajeros por mes en US en todas las aerolíneas.
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El flujo de pasajeros es muy irregular a lo largo de todo el período, lo cual indica una gran fluctuación mensual a lo largo del período. 

Se aprecia un crecimiento continuo hasta alcanzar el pico a mediados de año. Luego inicia el descenso con otros saltos en octubre y diciembre.

La media móvil refleja la tendencia del flujo ascendente de pasajeros.
</div>

In [66]:
# Comportamiento del flujo de pasajeros en US y el promedio movil
df_3_us = process_passengers_month(df_1, "GEO Region", ['US',])
plot_passengers_month(df_3_us, "GEO Region", ['US',], roll_mean='Y')


![](newplot17.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Total de pasajeros por mes en todas las regiones y aerolíneas, excepto US. 
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El flujo de pasajeros por región es bastante dispar. Asia, Europa y Canadá concentran la mayor cantidad de pasajeros sin incluir Estados Unidos. En el resto de regiones las cifras resultan mínimas en comparación con las anteriores, aunque Mexico alcanza unos picos de mas de 100 mil pasajeros en julio del 2014 y julio y diciembre del 2015. 
En Australia/Oceania, Middle East, Central America y South America, la media móvil casi coincide con el total pasajeros, lo cual indica que existe poca variación en el flujo de pasajeros en esas rutas.
</div>

In [67]:
# Comportamiento del flujo de pasajeros en todas las regiones, excepto US y el promedio movil en cada una de ellas
sel_regions = region.copy()
sel_regions.remove('US')
df_3_sel_reg = process_passengers_month(df_1, "GEO Region", sel_regions)
plot_passengers_month(df_3_sel_reg, "GEO Region", sel_regions, roll_mean='Y') 

![](newplot18.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
. Generación de Gráfico de Pasajeros por Mes con Opciones de Filtrado y Media Móvil mensual para todo el período 2005-2016.
</div>

In [68]:
# Función para generar gráfico de líneas para analizar los pasajeros por mes y año
def passengers_month_year(df_1, columna, seleccion=['todo'], columna_data='passengers', ancho=1000, alto=600):
    """
    Genera un gráfico de líneas que muestra los pasajeros por mes y año.

    Parámetros:
    - df_1: DataFrame modificado con los datos de pasajeros.
    - columna: Columna para filtrar los datos (ej. 'Operating Airline' o 'GEO Region').
    - seleccion: Lista de valores para filtrar en la columna especificada. Por defecto, incluye todos los valores.
    - columna_data: Nombre de la columna que contiene los datos de pasajeros. Por defecto, 'passengers'.
    - ancho: Ancho del gráfico. Por defecto, 1000.
    - alto: Alto del gráfico. Por defecto, 600.
    """
    # Definir las columnas relacionadas
    columna1, columna2 = 'Operating Airline', 'GEO Region'
    otra_columna = columna2 if columna == columna1 else columna1

    # Filtrar datos según la selección
    df_2 = df_1 if seleccion == ['todo'] else df_1.filter(F.col(columna).isin(seleccion))

    # Agregar y agrupar por fecha
    df_3 = df_2.groupBy('Date').agg(F.sum(columna_data).alias(columna_data))
    
    # Obtener los años únicos ordenados
    year_ = (
        df_3.select(F.year('Date').alias('Year'))
        .distinct()
        .orderBy('Year')
        .rdd.flatMap(lambda x: x)
        .collect()
    )

    # Preparar los datos para el gráfico
    plot_data = []
    for anyo in year_:
        # Filtrar los datos por año y ordenar por mes
        df_4 = df_3.filter(F.year('Date') == anyo).orderBy(F.month('Date'))
        
        # Extraer los meses y datos de pasajeros
        data = df_4.select(F.month('Date').alias('Month'), columna_data).collect()
        plot_data.extend({'Year': anyo, 'Month': row['Month'], columna_data: row[columna_data]} for row in data)

    # Convertir los datos a un DataFrame de Pandas
    plot_df = pd.DataFrame(plot_data)

    # Crear el gráfico con plotly.express
    fig = px.line(
        plot_df,
        x='Month',
        y=columna_data,
        color='Year',
        labels={'Month': 'Mes', columna_data: 'Pasajeros', 'Year': 'Año'},
        title=(
            f"Pasajeros por mes, {', '.join(map(str, seleccion))} {columna}(s) y todas {otra_columna}(s)"
            if len(seleccion) <= 3
            else f"Pasajeros por mes y por {columna} (selección múltiple) y todo {otra_columna}(s)"
        )
    )

    # Ajustar el diseño del gráfico
    fig.update_layout(
        width=ancho,
        height=alto,
        plot_bgcolor='#FBFCFF',
        title_font_size=20,
        xaxis_title='Mes',
        yaxis_title='Pasajeros',
        legend_title='Año'
    )

    # Mostrar el gráfico
    fig.show()


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
La demanda de pasajeros cae en febrero, aumenta en marzo, se mantiene en abril y sigue creciendo hasta agosto. De nuevo se produce una caida en septiembre, aumenta en octubre, cae en noviembre y de nuevo repunta en diciembre.
</div>

In [69]:
# Comportamiento mensual del flujo de pasajeros en todas las aerolíneas durante 2005-2016
df_3_year = passengers_month_year(df_1, 'GEO Region') 

![](newplot19.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
Comportamiento de United Airlines
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
Se observa un salto brusco en 2013, lo cual está relacionado con la existencia de dos entradas en el dataframe para esta aerolínea. United Airlines es la aerolínea que transporta la mayor cantidad de pasajeros en rutas nacionales e internacionales.
</div>

In [70]:
# Comportamiento del flujo de pasajeros de la aerolínea con mayor número de pasajeros en el dataframe
df_3_ua = passengers_month_year(df_1, 'Operating Airline', ['United Airlines']) 

![](newplot20.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
Comportamiento de la línea europea y asiática que transporta la mayor cantidad de pasajeros.
</div>

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
<b>Europa</b> Lufthansa German Airlines muestra entre 2011 y 2015 una tendencia de crecimiento sostenido entre los meses de febrero a septiembre, dibujando una especie de semicírculo. Los años previos la curva es casi plana con una fuerte caída en el mes de febrero. En 2010 la caída es mas pronunciada.
</div>

In [71]:
# Principal aerolínea europea por el total de pasajeros transportados
df_3_lufthansa = passengers_month_year(df_1, 'Operating Airline', ['Lufthansa German Airlines'])

![](newplot21.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
<b>Asia</b> A partir del 2008 en adelante, Cathay Pacific muestra un flujo estable de pasajeros. El comportamiento mensual refleja que febrero y septiembre son los meses con menor movimiento. Llama la atención el salto brusco ocurrido en septiembre del 2007. Antes de esa fecha, el total de pasajeros está en torno a 22 mil pasajeros mensuales. Luego da un salto hasta alcanzar la cifra de más de 38 mil pasajeros en diciembre. 
</div>

In [72]:
# Principal aerolínea asiática por el total de pasajeros transportados
df_3_cathay = passengers_month_year(df_1, 'Operating Airline', ['Cathay Pacific']) 

![](newplot22.png)

<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
Regresión Lineal
</div>

In [73]:
#df_3 = process_passengers_month(df_1, "GEO Region", ['todo'])

# Obtener la fecha mínima en la columna 'Date'
min_date = df_3_all.agg(F.min("Date")).collect()[0][0] 

# Calcular la columna 'Time' que represente la diferencia en meses entre cada fecha en Date y la fecha mínima (min_date)
df_3_linear = df_3_all.withColumn('Time', (F.unix_timestamp('Date') - F.unix_timestamp(F.lit(min_date))) / 60 / 60 / 24 / 30)  # Diferencia en meses


In [74]:
# Agregar una columna constante (intercepto)
df_3_linear = df_3_linear.withColumn('constant', F.lit(1))

# Crear el vector de características (X)
assembler = VectorAssembler(inputCols=['Time', 'constant'], outputCol='features')
df_3_linear = assembler.transform(df_3_linear)

# Ajustar el modelo de regresión lineal
lr = LinearRegression(featuresCol='features', labelCol='passengers')
lr_model = lr.fit(df_3_linear)

# Calcular predicciones
predictions = lr_model.transform(df_3_linear)

# Crear evaluador de regresión
evaluator = RegressionEvaluator(labelCol="passengers", predictionCol="prediction")

# Calcular métricas
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})  # Root Mean Squared Error
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})  # Mean Absolute Error
mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})  # Mean Squared Error
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})  # Coeficiente de determinación (R²)


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
El coeficiente de determinación es menor a 60%, lo cual indica que el modelo es poco efectivo. 
</div>

In [75]:
# Mostrar métricas
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² (Coeficiente de determinación): {r2:.2f}")

Root Mean Squared Error (RMSE): 359202.61
Mean Absolute Error (MAE): 282246.92
Mean Squared Error (MSE): 129026513439.42
R² (Coeficiente de determinación): 0.59


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
.Realizar la predicción.
</div>


In [76]:
# Realizar la predicción con el modelo ajustado
df_3_linear = lr_model.transform(df_3_linear)

# La predicción se guarda en una nueva columna
df_3_linear = df_3_linear.withColumnRenamed("prediction", "passengers_")

# Imprimir las columnas de 'linear'
print(df_3_linear.columns)



['Date', 'passengers', 'Time', 'constant', 'features', 'passengers_']


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 14px;">
.Extender la predicción hasta 2020.
</div>

In [77]:

# Extender la predicción hasta 2020 (periods=60)
df_3_predict_pd = pd.DataFrame(data={
    'Date': pd.date_range('2016-03-01', periods=60, freq='M') + pd.DateOffset(days=1),
    'passengers': np.nan
})

# Convertir el DataFrame de pandas a PySpark
spark = SparkSession.builder.getOrCreate()
df_3_predict = spark.createDataFrame(df_3_predict_pd)

# Convertir la columna 'Date' a tipo 'date' 
df_3_predict = df_3_predict.withColumn('Date', F.col('Date').cast('date'))

# Calcular la columna 'Time' para las fechas futuras
min_date = df_3_linear.agg(F.min("Date")).collect()[0][0]
df_3_predict = df_3_predict.withColumn('Time', (F.unix_timestamp('Date') - F.unix_timestamp(F.lit(min_date))) / 60 / 60 / 24 / 30)

# Agregar la columna constante y preparar características para la predicción
df_3_predict = df_3_predict.withColumn('constant', F.lit(1))
df_3_predict_assembler = VectorAssembler(inputCols=['Time', 'constant'], outputCol='features')
df_3_predict = df_3_predict_assembler.transform(df_3_predict)

# Eliminar la columna 'passengers_' si ya existe antes de aplicar la predicción
df_3_predict = df_3_predict.drop('passengers_')

# Realizar la predicción
df_3_predict = lr_model.transform(df_3_predict)

# Renombrar la columna de predicción a 'passengers_'
df_3_predict = df_3_predict.withColumnRenamed("prediction", "passengers_")

# Imprimir las columnas de 'predict'
print(df_3_predict.columns)


['Date', 'passengers', 'Time', 'constant', 'features', 'passengers_']


In [78]:

# Crear los DataFrames necesarios para graficar
df_3_linear_pd = df_3_linear.select('Date', 'passengers_').toPandas()
df_3_predict_pd = df_3_predict.select('Date', 'passengers_').toPandas()

In [79]:
def plot_line_chart(dataframes, x_col, y_col_list, width=1000, height=600, y_label='Pasajeros', title='', color_cycle=None):
    """
    Función para graficar múltiples series de datos en un gráfico de líneas.
    
    Parámetros:
    - dataframes (list): Lista de DataFrames de Spark que contienen los datos.
    - x_col (str): Nombre de la columna que se usará en el eje X.
    - y_col_list (list of lists): Lista de listas de nombres de columnas que se graficarán.
    - width (int, optional): Ancho del gráfico. Por defecto 1000.
    - height (int, optional): Alto del gráfico. Por defecto 600.
    - y_label (str, optional): Etiqueta del eje Y. Por defecto 'Pasajeros'.
    - title (str, optional): Título del gráfico.
    - color_cycle (list, optional): Lista de colores para las líneas. Si no se proporciona, se usa un ciclo predeterminado.
    
    Retorna:
    - None (muestra el gráfico).
    """

    # Definir un ciclo de colores predeterminado si no se proporciona
    if color_cycle is None:
        color_cycle = ['orange', 'navy', 'tomato', 'crimson', 'darkgreen', 'brown', 'forestgreen', 
                       'steelblue', 'deeppink', 'darkorange', 'palegreen', 'gold', 'darkviolet']
    
    traces = []
    color_index = 0
    
    # Iterar sobre los DataFrames y las columnas de interés
    for df_idx, df_1 in enumerate(dataframes):
        for y_col in y_col_list[df_idx]:
            # Renombrar las columnas para evitar ambigüedades
            df_1_renamed = df_1.select([F.col(x_col)] + [F.col(y_col).alias(f'{y_col}_{df_idx}')])
            
            # Convertir a pandas para graficar
            x_data = df_1_renamed.select(x_col).toPandas()[x_col]
            y_data = df_1_renamed.select(f'{y_col}_{df_idx}').toPandas()[f'{y_col}_{df_idx}']
            
            # Agregar la traza con las líneas
            traces.append(go.Scatter(
                x=x_data,
                y=y_data,
                mode='lines',
                line=dict(color=color_cycle[color_index]),
                name=f'{y_label} {df_idx+1}'  # Cambiar la leyenda según el índice del DataFrame
            ))
            
            # Actualizar el índice de color
            color_index = (color_index + 1) % len(color_cycle)
    
    # Crear la figura
    fig = go.Figure(traces)
    
    # Configurar el layout del gráfico
    fig.update_layout(
        title=title,
        xaxis_title='Año',  # Cambiar "date" por "Año"
        yaxis_title=y_label,
        plot_bgcolor='#F7FAFC',
        title_font_size=20,
        xaxis_title_font=dict(size=15),
        yaxis_title_font=dict(size=15),
        showlegend=True,
        width=width,
        height=height
    )
    
    # Mostrar el gráfico
    fig.show()


<div style="letter-spacing: 0.5px; text-align: justify; font-size: 12px;">
  La <b style="color: orange;">línea naranja</b> representa la cantidad de pasajeros por año. Su tendencia es creciente, con fluctuaciones anuales marcadas debido a estaciones y/o eventos específicos.

  La <b style="color: blue;">línea azul</b> muestra las predicciones del modelo de regresión lineal para los pasajeros. Esta línea también sigue una tendencia creciente, pero con menor variabilidad que la <b style="color: orange;">línea naranja</b>, lo que sugiere que el modelo ofrece una aproximación más estable de la evolución del número de pasajeros.

  Por su parte, la <b style="color: red;">línea roja</b> proyecta la tendencia general del crecimiento de los pasajeros a partir del 2016. Sin embargo, al igual que la <b style="color: blue;">línea azul</b> no refleja las fluctuaciones interanuales representadas por la <b style="color: orange;">línea naranja</b>. Es importante recordar que, a finales del 2019, se detectó el virus COVID-19, el cual se prpagó rápidamente a nivel global, afectando a la humanidad y dejando repercusiones económicas que aún hoy perduran.
</div>


In [80]:
# # Generar el gráfico con los DataFrames corregidos
plot_line_chart([df_3_linear, df_3_predict], 'Date', \
               [['passengers', 'passengers_'], ['passengers_']], title='Movimiento Anual de pasajeros y su proyección hasta el 2020')

![](newplot23.png)